In [1]:
!pip install paddle-quantum

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 71kB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 208.1MB 8.9MB/s eta 0:00:014     |█████████████████████████████▏  | 189.6MB 8.3MB/s eta 0:00:03
  Created wheel for interval: filename=interval-1.0.0-cp37-none-any.whl size=14266 sha256=eb5b25225c29525a4702fbe738a1932479e55b47a9b0948c437dec1aad172a79
  Stored in directory: /home/aistudio/.cache/pip/wheels/aa/fd/f0/28a65289ea3a10ea744cfe6d2393605383772be2ab2b75967e
Successfully built interval
  Found existing installation: paddlepaddle 1.8.4
    Uninstalling paddlepaddle-1.8.4:
      Successfully uninstalled paddlepaddle-1.8.4


In [2]:
import numpy as np
import paddle
from paddle import matmul, transpose, trace
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import dagger, random_pauli_str_generator, pauli_str_to_matrix
from paddle_quantum.state import vec, vec_random, density_op, density_op_random

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
2021-03-13 23:06:43,180 - INFO - font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/co

In [3]:
import math
u = np.array([[1,-1],
             [1, 1]])
u = 1/math.sqrt(2)*u
print(u)

theta = np.array([np.pi])
theta = paddle.to_tensor(theta)
num_qubits = 1
cir = UAnsatz(num_qubits)

which_qubit = 0
cir.ry(theta,which_qubit)

print(cir.U.numpy())
x = np.transpose(cir.U.numpy().real)
x = np.matmul(u,x)
print(x)
F = 1/2*x.trace()
print(F)

[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
[[ 6.123234e-17+0.j -1.000000e+00+0.j]
 [ 1.000000e+00+0.j  6.123234e-17+0.j]]
[[ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]
0.7071067811865475


In [2]:
#单量子比特近似

import math
import numpy as np
import paddle
from paddle import matmul, transpose, trace
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import dagger, random_pauli_str_generator, pauli_str_to_matrix
from paddle_quantum.state import vec, vec_random, density_op, density_op_random
theta_size = 1
num_qubits = 1
ITR = 200
LR=0.5
SEED = 1
paddle.seed(SEED)

def U_theta(theta,num_qubits):
    cir = UAnsatz(num_qubits)
    which_qubit = 0
    cir.ry(theta[0],which_qubit)
    return cir.U

class Optimization_exl(paddle.nn.Layer):
    def __init__(self,shape,dtype='float64'):
        super(Optimization_exl,self).__init__()
        self.u = paddle.to_tensor(1/math.sqrt(2)* np.array([[1,-1],[1, 1]]))
        self.theta = self.create_parameter(shape=shape,
                                            default_initializer = paddle.nn.initializer.Uniform(low=0,high=2*np.pi),
                                            dtype=dtype,is_bias=False)
    
    def forward(self):
        U = U_theta(self.theta,num_qubits)
        U_dagger = dagger(U)
        loss = 1-(0.5*paddle.real(trace(matmul(self.u,U_dagger)))[0])
        return loss
    
    def result(self):
        return self.theta

loss_list = []
parameter_list = []
myLayer = Optimization_exl([theta_size])
opt = paddle.optimizer.Adam(learning_rate=LR,parameters=myLayer.parameters())

for itr in range(ITR):
    loss = myLayer()[0]
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    loss_list.append(loss.numpy()[0])
    parameter_list.append(myLayer.parameters()[0].numpy())
    if itr % 5 == 0:
        print('iter:', itr, '  loss: %.4f' % loss.numpy())
print(loss_list[-1])
print(myLayer.result())

iter: 0   loss: 0.2543
iter: 5   loss: 0.0453
iter: 10   loss: 0.0139
iter: 15   loss: 0.0211
iter: 20   loss: 0.0027
iter: 25   loss: 0.0088
iter: 30   loss: 0.0002
iter: 35   loss: 0.0036
iter: 40   loss: 0.0001
iter: 45   loss: 0.0010
iter: 50   loss: 0.0004
iter: 55   loss: 0.0001
iter: 60   loss: 0.0003
iter: 65   loss: 0.0000
iter: 70   loss: 0.0000
iter: 75   loss: 0.0001
iter: 80   loss: 0.0000
iter: 85   loss: 0.0000
iter: 90   loss: 0.0000
iter: 95   loss: 0.0000
iter: 100   loss: 0.0000
iter: 105   loss: 0.0000
iter: 110   loss: 0.0000
iter: 115   loss: 0.0000
iter: 120   loss: 0.0000
iter: 125   loss: 0.0000
iter: 130   loss: 0.0000
iter: 135   loss: 0.0000
iter: 140   loss: 0.0000
iter: 145   loss: 0.0000
iter: 150   loss: 0.0000
iter: 155   loss: 0.0000
iter: 160   loss: 0.0000
iter: 165   loss: 0.0000
iter: 170   loss: 0.0000
iter: 175   loss: 0.0000
iter: 180   loss: 0.0000
iter: 185   loss: 0.0000
iter: 190   loss: 0.0000
iter: 195   loss: 0.0000
3.739453191542452e-11


In [10]:
#双量子门

import math
import numpy as np
import paddle
from paddle import matmul, transpose, trace
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import dagger, random_pauli_str_generator, pauli_str_to_matrix
from paddle_quantum.state import vec, vec_random, density_op, density_op_random
theta_size = 4
num_qubits = 2
ITR = 80
LR=0.5
SEED = 1
paddle.seed(SEED)




def U_theta(theta,num_qubits):
    cir = UAnsatz(num_qubits)
    cir.ry(theta[0],0)
    cir.ry(theta[1],1)
    cir.cnot([0,1])
    cir.ry(theta[2],0)
    cir.ry(theta[3],1)
    return cir.U

class Optimization_exl(paddle.nn.Layer):
    def __init__(self,shape,dtype='float64'):
        super(Optimization_exl,self).__init__()
        self.u = paddle.to_tensor(np.array([[0.098757, -0.305705, 0.207859, 0.923897],
                                            [-0.112903, 0.177275, 0.966589, -0.146738],
                                            [-0.795313, 0.514071, -0.143547, 0.287407],
                                            [-0.587348, -0.781568, 0.043521, -0.205619 ]]))
        self.theta = self.create_parameter(shape=shape,
                                            default_initializer = paddle.nn.initializer.Uniform(low=0,high=2*np.pi),
                                            dtype=dtype,is_bias=False)
    
    def forward(self):
        U = U_theta(self.theta,num_qubits)
        U_dagger = dagger(U)
        loss = 1-(0.25*paddle.real(trace(matmul(self.u,U_dagger)))[0])
        return loss
    
    def result(self):
        return self.theta

loss_list = []
parameter_list = []
myLayer = Optimization_exl([theta_size])
opt = paddle.optimizer.Adam(learning_rate=LR,parameters=myLayer.parameters())

for itr in range(ITR):
    loss = myLayer()[0]
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    loss_list.append(loss.numpy()[0])
    parameter_list.append(myLayer.parameters()[0].numpy())
    if itr % 5 == 0:
        print('iter:', itr, '  loss: %.4f' % loss.numpy())
print(loss_list[-1])
print(myLayer.result())

ImportError: cannot import name 'matmul' from 'paddle' (/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/__init__.py)

In [51]:
#三量子门

import math
import numpy as np
import paddle
from paddle import matmul, transpose, trace
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import dagger, random_pauli_str_generator, pauli_str_to_matrix
from paddle_quantum.state import vec, vec_random, density_op, density_op_random
theta_size = 5
num_qubits = 3
ITR = 100
LR=0.5
SEED = 1
paddle.seed(SEED)
path = '飞桨常规赛：量子电路合成/Question_3_Unitary.txt'
def readfile(path):
    f = np.loadtxt(path)
    return f

def U_theta(theta,num_qubits):
    cir = UAnsatz(num_qubits)
    cir.ry(theta[0],0)
    cir.ry(theta[1],1)
    cir.ry(theta[2],2)
    cir.cnot([0,1])
    cir.cnot([1,2])
    cir.ry(theta[3],0)
    cir.ry(theta[4],2)
    return cir.U

class Optimization_exl(paddle.nn.Layer):
    def __init__(self,shape,dtype='float64'):
        super(Optimization_exl,self).__init__()
        f = readfile(path)
        self.u = paddle.to_tensor(f)
        self.theta = self.create_parameter(shape=shape,
                                            default_initializer = paddle.nn.initializer.Uniform(low=0,high=2*np.pi),
                                            dtype=dtype,is_bias=False)
    
    def forward(self):
        U = U_theta(self.theta,num_qubits)
        U_dagger = dagger(U)
        loss = 1-(0.125*paddle.real(trace(matmul(self.u,U_dagger)))[0])
        return loss
    
    def result(self):
        return self.theta

loss_list = []
parameter_list = []
myLayer = Optimization_exl([theta_size])
opt = paddle.optimizer.Adam(learning_rate=LR,parameters=myLayer.parameters())

for itr in range(ITR):
    loss = myLayer()[0]
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    loss_list.append(loss.numpy()[0])
    parameter_list.append(myLayer.parameters()[0].numpy())
    if itr % 5 == 0:
        print('iter:', itr, '  loss: %.4f' % loss.numpy())
print(loss_list[-1])
print(myLayer.result())

iter: 0   loss: 0.8550
iter: 5   loss: 0.0763
iter: 10   loss: 0.1011
iter: 15   loss: 0.0299
iter: 20   loss: 0.0472
iter: 25   loss: 0.0014
iter: 30   loss: 0.0188
iter: 35   loss: 0.0034
iter: 40   loss: 0.0048
iter: 45   loss: 0.0010
iter: 50   loss: 0.0026
iter: 55   loss: 0.0004
iter: 60   loss: 0.0006
iter: 65   loss: 0.0002
iter: 70   loss: 0.0002
iter: 75   loss: 0.0002
iter: 80   loss: 0.0000
iter: 85   loss: 0.0001
iter: 90   loss: 0.0000
iter: 95   loss: 0.0000
1.5951791803470172e-05
Parameter containing:
Tensor(shape=[5], dtype=float64, place=CPUPlace, stop_gradient=False,
       [-2.31120880,  4.50132263,  5.94368069, -1.46651638,  1.98734749])


In [60]:
#三量子比特门无结构分解

import math
import numpy as np
import paddle
#from paddle import matmul, transpose, trace
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import dagger, random_pauli_str_generator, pauli_str_to_matrix
from paddle_quantum.state import vec, vec_random, density_op, density_op_random
theta_size = 12
num_qubits = 3
ITR =100
LR=0.5
SEED = 1
paddle.seed(SEED)
path = '飞桨常规赛：量子电路合成/Question_4_Unitary.txt'
def readfile(path):
    f = np.loadtxt(path)
    return f

def U_theta(theta,num_qubits):
    cir = UAnsatz(num_qubits)
    cir.ry(theta[0],0)
    cir.ry(theta[1],1)
    cir.ry(theta[2],2)    
    cir.cnot([0,1])
    cir.cnot([1,2])
    cir.ry(theta[3],0)
    cir.ry(theta[4],1)
    cir.ry(theta[5],2)
    cir.cnot([0,1])
    cir.cnot([1,2])
    cir.ry(theta[6],0)
    cir.ry(theta[7],1)
    cir.ry(theta[8],2)
    cir.cnot([0,1])
    cir.cnot([1,2])
    cir.ry(theta[9],0)
    cir.ry(theta[10],1)
    cir.ry(theta[11],2)
    cir.cnot([0,1])
    cir.cnot([1,2])

    return cir.U

class Optimization_exl(paddle.nn.Layer):
    def __init__(self,shape,dtype='float64'):
        super(Optimization_exl,self).__init__()
        f = readfile(path)
        self.u = paddle.to_tensor(f)
        self.theta = self.create_parameter(shape=shape,
                                            default_initializer = paddle.nn.initializer.Uniform(low=0,high=2*np.pi),
                                            dtype=dtype,is_bias=False)
    
    def forward(self):
        U = U_theta(self.theta,num_qubits)
        U_dagger = dagger(U)
        loss = 1-(0.125*paddle.real(trace(matmul(self.u,U_dagger)))[0])
        return loss
    
    def result(self):
        return self.theta

loss_list = []
parameter_list = []
myLayer = Optimization_exl([theta_size])
opt = paddle.optimizer.Adam(learning_rate=LR,parameters=myLayer.parameters())

for itr in range(ITR):
    loss = myLayer()[0]
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    loss_list.append(loss.numpy()[0])
    parameter_list.append(myLayer.parameters()[0].numpy())
    if itr % 5 == 0:
        print('iter:', itr, '  loss: %.4f' % loss.numpy())
print(loss_list[-1])
print(myLayer.result())

iter: 0   loss: 0.9774
iter: 5   loss: 0.5517
iter: 10   loss: 0.2901
iter: 15   loss: 0.2762
iter: 20   loss: 0.2880
iter: 25   loss: 0.2536
iter: 30   loss: 0.2330
iter: 35   loss: 0.2320
iter: 40   loss: 0.2322
iter: 45   loss: 0.2296
iter: 50   loss: 0.2279
iter: 55   loss: 0.2267
iter: 60   loss: 0.2261
iter: 65   loss: 0.2261
iter: 70   loss: 0.2260
iter: 75   loss: 0.2260
iter: 80   loss: 0.2259
iter: 85   loss: 0.2259
iter: 90   loss: 0.2259
iter: 95   loss: 0.2259
0.22584664419782585
Parameter containing:
Tensor(shape=[12], dtype=float64, place=CPUPlace, stop_gradient=False,
       [-0.34302257,  3.79840820,  4.89255843, -0.63279702,  4.65861761,  3.83682412,  4.54703008,  3.37265171,  1.35047891,  0.84161231,  3.96285257,  6.62242478])


In [97]:
#四量子比特门无结构分解

import math
import numpy as np
import paddle
#from paddle import matmul, transpose, trace
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import dagger, random_pauli_str_generator, pauli_str_to_matrix
from paddle_quantum.state import vec, vec_random, density_op, density_op_random
theta_size = 40
num_qubits = 4
ITR =100
LR=0.5
SEED = 1
paddle.seed(SEED)
path = '飞桨常规赛：量子电路合成/Question_5_Unitary.txt'
def readfile(path):
    f = np.loadtxt(path)
    return f

def U_theta(theta,num_qubits):
    cir = UAnsatz(num_qubits)
    for i in range(10):
        for j in range(4):
            cir.ry(theta[4*i+j],j)
            if j<3:
                cir.cnot([j+1,j])
        #for j in range(3):
            #cir.cnot([j,(j+1)])
    return cir.U

class Optimization_exl(paddle.nn.Layer):
    def __init__(self,shape,dtype='float64'):
        super(Optimization_exl,self).__init__()
        f = readfile(path)
        self.u = paddle.to_tensor(f)
        self.theta = self.create_parameter(shape=shape,
                                            default_initializer = paddle.nn.initializer.Uniform(low=0,high=2*np.pi),
                                            dtype=dtype,is_bias=False)
    
    def forward(self):
        U = U_theta(self.theta,num_qubits)
        U_dagger = dagger(U)
        loss = 1-(0.5*0.125*paddle.real(trace(matmul(self.u,U_dagger)))[0])
        return loss
    
    def result(self):
        return self.theta

loss_list = []
parameter_list = []
myLayer = Optimization_exl([theta_size])
opt = paddle.optimizer.Adam(learning_rate=LR,parameters=myLayer.parameters())

for itr in range(ITR):
    loss = myLayer()[0]
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    loss_list.append(loss.numpy()[0])
    parameter_list.append(myLayer.parameters()[0].numpy())
    if itr % 5 == 0:
        print('iter:', itr, '  loss: %.4f' % loss.numpy())
print(loss_list[-1])
print(myLayer.result())

iter: 0   loss: 0.9833
iter: 5   loss: 0.6137
iter: 10   loss: 0.5062
iter: 15   loss: 0.4351
iter: 20   loss: 0.3789
iter: 25   loss: 0.3600
iter: 30   loss: 0.3493
iter: 35   loss: 0.3422
iter: 40   loss: 0.3292
iter: 45   loss: 0.3246
iter: 50   loss: 0.3169
iter: 55   loss: 0.2998
iter: 60   loss: 0.2780
iter: 65   loss: 0.2746
iter: 70   loss: 0.2611
iter: 75   loss: 0.2532
iter: 80   loss: 0.2538
iter: 85   loss: 0.2520


In [73]:
!zip Answer.zip Question_*

updating: Question_1_Answer.txt (stored 0%)
updating: Question_2_Answer.txt (deflated 11%)
updating: Question_3_Answer.txt (deflated 22%)
updating: Question_4_Answer.txt (deflated 51%)
updating: Question_5_Answer.txt (stored 0%)
updating: Question_6_Answer.txt (stored 0%)
